In [1]:
import warnings
warnings.filterwarnings("ignore")  # Ignore all warnings
from typing import List, Dict, Any

import numpy as np
# import matplotlib.pyplot as plt
from agentic_energy.language_models.basic_llm_amap import records_to_arrays, llm_solve, llm_solve_from_records, solve_daily_llm
from agentic_energy.data_loader import EnergyDataLoader, BatteryDataLoader
from agentic_energy.schemas import BatteryParams, DayInputs, EnergyDataRecord, SolveRequest, SolveFromRecordsRequest

Forecast Engine using device: cpu


#### Forecast Data Load

In [2]:
italy_forecast_obj = EnergyDataLoader(region = "ITALY", data_version="forecast", forecast_type="RF")
italy_forecast_data = italy_forecast_obj.load_region_data()
italy_forecast = await italy_forecast_obj.get_filtered_data("2018-01-01", "2018-01-01")

2025-11-18 00:57:43.385 | DEBUG    | agentics.core.agentics:from_csv:307 - Importing Agentics of type EnergyDataRecord from CSV c:\Users\16467\OneDrive\Desktop\Columbia\Agentics\Another\Agentics_for_EnergyArbitrage_Battery\energy_arbitrage\agentic_energy\data\Italy_data_forecast_RF.csv
2025-11-18 00:57:43.455 | DEBUG    | agentics.core.llm_connections:get_llm_provider:47 - Available LLM providers: ['gemini', 'openai', 'ollama']. None specified, defaulting to 'ollama'
2025-11-18 00:57:43.560 | DEBUG    | agentics.core.llm_connections:get_llm_provider:47 - Available LLM providers: ['gemini', 'openai', 'ollama']. None specified, defaulting to 'ollama'


In [3]:
stats_forecast_consumption = await EnergyDataLoader.get_summary_stats_from_ag(italy_forecast_data, column="consumption")
print(stats_forecast_consumption.pretty_print())

2025-11-18 00:57:43.571 | DEBUG    | agentics.core.llm_connections:get_llm_provider:47 - Available LLM providers: ['gemini', 'openai', 'ollama']. None specified, defaulting to 'ollama'


aType : <class 'agentic_energy.schemas.MetricStats'>
count: 24
min: 21.71582122
max: 34.94844843
avg: 27.93677364625
median: 28.378208385
p25: 24.5850403875
p75: 30.0580908925
std: 3.9238253542943533
var: 15.396405411003206


aType : <class 'agentic_energy.schemas.MetricStats'>
count: 24
min: 21.71582122
max: 34.94844843
avg: 27.93677364625
median: 28.378208385
p25: 24.5850403875
p75: 30.0580908925
std: 3.9238253542943533
var: 15.396405411003206




##### Actual Data Load

In [4]:
italy_actual_obj = EnergyDataLoader(region = "ITALY", data_version="actual")
italy_actual_data = italy_actual_obj.load_region_data()
italy_actual = await italy_actual_obj.get_filtered_data("2018-01-01", "2018-01-01")

2025-11-18 00:57:43.584 | DEBUG    | agentics.core.agentics:from_csv:307 - Importing Agentics of type EnergyDataRecord from CSV c:\Users\16467\OneDrive\Desktop\Columbia\Agentics\Another\Agentics_for_EnergyArbitrage_Battery\energy_arbitrage\agentic_energy\data\Italy_data_actual.csv
2025-11-18 00:57:43.813 | DEBUG    | agentics.core.llm_connections:get_llm_provider:47 - Available LLM providers: ['gemini', 'openai', 'ollama']. None specified, defaulting to 'ollama'
2025-11-18 00:57:43.905 | DEBUG    | agentics.core.llm_connections:get_llm_provider:47 - Available LLM providers: ['gemini', 'openai', 'ollama']. None specified, defaulting to 'ollama'


In [5]:
stats_actual_consumption = await EnergyDataLoader.get_summary_stats_from_ag(italy_actual_data, column="consumption")
print(stats_actual_consumption.pretty_print())

2025-11-18 00:57:43.926 | DEBUG    | agentics.core.llm_connections:get_llm_provider:47 - Available LLM providers: ['gemini', 'openai', 'ollama']. None specified, defaulting to 'ollama'


aType : <class 'agentic_energy.schemas.MetricStats'>
count: 24
min: 19.49
max: 30.97
avg: 24.374583333333334
median: 23.83
p25: 22.0275
p75: 25.564999999999998
std: 3.3568598659643545
var: 11.268508159722224


aType : <class 'agentic_energy.schemas.MetricStats'>
count: 24
min: 19.49
max: 30.97
avg: 24.374583333333334
median: 23.83
p25: 22.0275
p75: 25.564999999999998
std: 3.3568598659643545
var: 11.268508159722224




In [6]:
stats_forecast_consumption.states[0].p25

24.5850403875

#### Battery Config Load

In [7]:
batt = BatteryDataLoader(
    load_stats={
        "p25":stats_forecast_consumption.states[0].p25,
        "p75":stats_forecast_consumption.states[0].p75
    },
    duration_hours=4,
    soc_init=0.5,
    soc_min=0.0,
    soc_max=1.0,
    eta_c = 0.95,
    eta_d = 0.95,
    soc_target=0.5,
)
battery = batt.compute_battery_params()
print(batt.summary())

{'Capacity (kWh)': 21.89, 'Charge Power (kW)': 5.47, 'Discharge Power (kW)': 5.47, 'Efficiency (Charge/Discharge)': (0.95, 0.95), 'Duration (hours)': 4}


In [8]:
result_llm = []
day_info = []
decision_list = []
soc_list = []  # Initialize soc_list

for i in range(len(italy_forecast_data)//24):
    print(f"Solving day {i+1} of {len(italy_forecast_data)//24}")

    forecast_prices, forecast_demand = records_to_arrays(italy_forecast_data[i*24:(i+1)*24])
    actual_prices, actual_demand = records_to_arrays(italy_actual_data[i*24:(i+1)*24])
    day = DayInputs(
        prices_buy=actual_prices,
        demand_kw=actual_demand,
        prices_sell=actual_prices,
        allow_export=True,
        dt_hours=1.0,
        prices_buy_forecast=forecast_prices,
        demand_kw_forecast=forecast_demand,
        prices_sell_forecast=forecast_prices,
    )
    day_info.append(day)
    req = SolveRequest(
        battery=battery,
        day=day,
        solver=None,
        solver_opts=None
    )
    res = await solve_daily_llm(req)
    result_llm.append(res)
    decision_list += res.decision 
    battery.soc_init = res.soc[-1]
    soc_list += res.soc[:-1]

2025-11-18 00:57:43.992 | DEBUG    | agentics.core.llm_connections:get_llm_provider:47 - Available LLM providers: ['gemini', 'openai', 'ollama']. None specified, defaulting to 'ollama'
2025-11-18 00:57:43.992 | DEBUG    | agentics.core.llm_connections:get_llm_provider:47 - Available LLM providers: ['gemini', 'openai', 'ollama']. None specified, defaulting to 'ollama'


Solving day 1 of 1




╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: ff8b8a58-8465-4fbf-bb4f-31692dd1220c                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Task Executor                                                                                           │
│                                                                                                                 │
│  Task:                                                                                                          │
│  Your task is to transduce a source Pydantic Object into the specified Output type. Generate only slots that    │
│  are logically deduced from the input information, otherwise live then null.                                    │
│                                                                                                                 │
│  Read carefully the following instructions for executing your task:                                             │
│                                                                                                                 │
│          You are solving a daily battery scheduling optimization problem using forecast-based reasoning and     │
│  constraint satisfaction.                                                                                       │
│                                                                                                                 │
│          You are provided with both forecasted and actual market data:                                          │
│                                                                                                                 │
│          FORECAST INPUTS (for decision-making):                                                                 │
│              - Forecasted buying prices: [47.31896137 41.78432284 37.49251592 36.728515   36.24872824           │
│  39.56044182                                                                                                    │
│   43.50378076 51.54855095 55.71723352 51.20543591 51.3866384  55.1697128                                        │
│   47.17853383 48.61988    49.63520408 51.79375316 59.66892529 61.94714286                                       │
│   53.56690444 60.35956522 53.23160289 49.06159184 48.84639656 42.45350689]  (array of length T)                 │
│              - Forecasted selling prices: [47.31896137 41.78432284 37.49251592 36.728515   36.24872824          │
│  39.56044182                                                                                                    │
│   43.50378076 51.54855095 55.71723352 51.20543591 51.3866384  55.1697128                                        │
│   47.17853383 48.61988    49.63520408 51.79375316 59.66892529 61.94714286                                       │
│   53.56690444 60.35956522 53.23160289 49.06159184 48.84639656 42.45350689]  (array of length T)                 │
│              - Forecasted demand: [24.95664627 23.92997141 22.42978245 21.71582122 21.87623452 22.6034125       │
│   24.622      25.58646667 27.94575438 30.17360503 29.74489392 32.93638194                                       │
│   29.62439343 27.9        27.81625    28.81066239 29.7110531  30.01958618                                       │
│   34.9371681  34.94844843 32.71125    31.79713258 29.21149144 24.47416155]  (array of length T)                 │
│                                                                                                                 │
│          ACTUAL INPUTS (for ex-post evaluation):                                                                │
│              - Realized buying prices: [45.82 44.16 42.24 39.29 36.   41.99 42.25 44.97 45.   44.94 45.02       │
│  45.71                                                                                                          │
│   46.7  43.98 44.96 47.53 49.91 54.3  51.91 51.38 49.2 

Output()

...

Output()

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: f7dc99c8-24b5-47e0-b2c3-a0484e84e29d                                                                     │
│  Agent: Task Executor                                                                                           │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: ff8b8a58-8465-4fbf-bb4f-31692dd1220c                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output:                                                                                                  │
│  {"decision_kw":[0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-  │
│  1.0,0.0],"import_kw":[22.98,21.81,20.76,19.97,19.49,19.84,21.1,22.1,22.4,23.59,24.37,24.96,25.06,23.9,23.49,2  │
│  3.76,25.24,29.1,30.35,30.97,30.34,28.82,26.54,24.05],"export_kw":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  │
│  ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],"decision":[0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.  │
│  0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,0.0]}                                                    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


✓ Optimization successful
aType : <class 'agentic_energy.schemas.SolveResponse'>
status: success
message: null
objective_cost: 2458.79
charge_kw: null
discharge_kw: null
import_kw:
- 22.98
- 21.81
- 20.76
- 19.97
- 19.49
- 19.84
- 21.1
- 22.1
- 22.4
- 23.59
- 24.37
- 24.96
- 25.06
- 23.9
- 23.49
- 23.76
- 25.24
- 29.1
- 30.35
- 30.97
- 30.34
- 28.82
- 26.54
- 24.05
export_kw:
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
soc: null
decision:
- 0.0
- -1.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 1.74
- 29.1
- 30.35
- 30.97
- 30.34
- 28.82
- 26.54
- 24.05
confidence: null


aType : <class 'agentic_energy.schemas.SolveResponse'>
status: success
message: null
objective_cost: 2458.79
charge_kw: null
discharge_kw: null
import_kw:
- 22.98
- 21.81
- 20.76
- 19.97
- 19.49
- 19.84
- 21.1
- 22.1
- 22.4
- 23.59
- 24.37
- 24.96
- 25.06
- 23.9
- 23.49
- 23.76

TypeError: 'NoneType' object is not subscriptable